In [3]:
import pandas as pd
from pymongo import MongoClient

In [4]:
client = MongoClient('mongodb://localhost:27017')
db = client.mfers_collection
mfer = db.mfer

In [5]:
i = 0
for mf in mfer.find():
    i += 1
    if i == 1:
        df = pd.json_normalize(mf)
    else:
        temp = pd.json_normalize(mf)
        df = pd.concat([df, temp])

In [17]:
a = ['assetinfo']
d = dict(zip(a, df.asset_events))
d

{'assetinfo': [{'asset': {'id': 124152641,
    'num_sales': 0,
    'background_color': None,
    'image_url': 'https://lh3.googleusercontent.com/RGz3OJW6a0MwbZfOU1IVk1toOmwRVSXlci0hvAgHAOB6_OFykRjG-mRsmZoDFZSitxU3YhEDz15O6thQN8TemA4wR_BLqb_rRZut',
    'image_preview_url': 'https://lh3.googleusercontent.com/RGz3OJW6a0MwbZfOU1IVk1toOmwRVSXlci0hvAgHAOB6_OFykRjG-mRsmZoDFZSitxU3YhEDz15O6thQN8TemA4wR_BLqb_rRZut=s250',
    'image_thumbnail_url': 'https://lh3.googleusercontent.com/RGz3OJW6a0MwbZfOU1IVk1toOmwRVSXlci0hvAgHAOB6_OFykRjG-mRsmZoDFZSitxU3YhEDz15O6thQN8TemA4wR_BLqb_rRZut=s128',
    'image_original_url': 'ipfs://QmWmgfYhDWjzVheQyV2TnpVXYnKR25oLWCB2i9JeBxsJbz',
    'animation_url': None,
    'animation_original_url': None,
    'name': 'mfer #1',
    'description': 'mfers by sartoshi',
    'external_link': None,
    'asset_contract': {'address': '0x79fcdef22feed20eddacbb2587640e45491b757f',
     'asset_contract_type': 'non-fungible',
     'created_date': '2021-11-29T22:21:12.644606',
   